# Qiskit 설치 및 업데이트

최신 버젼의 Qiskit을 설치하는 것! 헷갈리신다면 다음의 순서를 따라해보세요 

1. [아나콘다 설치](https://www.anaconda.com/)

    1-1. 만약 오래된 아나콘다가 설치되어 있다면 터미널을 열어 다음의 명령어를 실행해 보세요

    `conda update -n base conda`
    
2. 환경 만들기

`conda create -n qiskit python=3.10`

3. 쥬피터 노트북 혹은 쥬피터 랩 설치

`conda install jupyter notebook` 혹은 `conda install jupyterlab`

4. qiskit_ibm_provider 설치

`pip install qiskit qiskit_ibm_provider`

5. qiskit-ibm-runtime 설치

`pip install qiskit-ibm-runtime`

6.  Qiskit 설치

`pip install qiskit`

6.  Qiskit 업데이트

`pip install -U qiskit`

설치가 잘 되었는지 다음의 명령어를 실행해서 확인해 봅시다.

In [4]:
import qiskit.tools.jupyter
%qiskit_version_table

## Qiskit Update News (6월 30일자 업데이트)

Qiskit 0.43 주요 업데이트 사항
- Opflow와 Quantum Instance를 더이상 사용할 수 없습니다: Opflow는 해밀토니안 시뮬레이션에, Quantum instance는 백엔드에 양자 회로를 실행하는데 중요한 역할을 해왔지만 이제 더 사용할 수 없게 되었습니다. 이 결정은 `qiskit.primitive` 를 중심으로 Qiskit의 효율성을 끌어올리고 코드 구조를 통일하면서 발생하게 되었습니다

<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*f3CGKqQksPN1hrmI1LmojA.png">
    - `qiskit.opflow.gradients` 는 `qiskit.algorithms.gradients` 로 대체 되며 다음과 같이 큰폭의 성능향상을 얻게 됩니다.
    
<img src = "https://miro.medium.com/v2/resize:fit:1400/format:webp/1*WOpP2SQOEqvS3kPjNeJWtQ.png">


   - [Operator Global](https://qiskit.org/documentation/stable/0.43/apidoc/opflow.html#operator-globals)을 더이상 사용할 수 없습니다.
   - 기존 사용자를 위한 대체 코드는 아래에 제공됩니다.
    
- nature, machine-learning, finance, optimization이 공식 Qiskit 패키지로 관리되지 않게 되었습니다. .

- Qiskit Terra가 0.25 로 업데이트 되었습니다.
    - 트랜스파일러가 끊어져있거나 여러개의 칩으로 구성된 장치를 지원하기 시작합니다: IBM의 하드웨어 개발 로드맵 중 "Heron"과 같은 형태의 분리되어 있는 QPU를 실시간 고전통신으로 연결하는 구조를 지원하기 시작합니다. 
    - [기타 업데이트 사항](https://github.com/Qiskit/qiskit-terra/milestones)
    

이어지는 실습에서 주요 업데이트와 관련하여 대체 코드등을 확인해 봅시다.

In [5]:
## qiskit-ibmq-provider alternative
from qiskit import *

from qiskit import QuantumCircuit
from qiskit_ibm_provider import IBMProvider

#IBMProvider.save_account(TOKEN)
#provider = IBMProvider()

#밋업을 위해 추가된 허브와 그룹, 프로젝트를 프로바이더에 설정
provider = IBMProvider(instance="ibm-q-yonsei/externalq-meetup/tutorials")

In [6]:
#밋업 참가자들에게 제공되는 양자 백엔드 목록
provider.backends()

[<IBMBackend('ibm_lagos')>,
 <IBMBackend('ibm_nairobi')>,
 <IBMBackend('ibm_perth')>]

## Operator Global

우선 가장 먼저, Operator Global의 변화를 확인해 봅시다. 업데이트 이전에 특정 양자게이트, 자주 사용하는 연산등을 사전에 정의하여 사용하는 것이 가능했지만 이제 다음과 같이 새로운 방식의 명령어를 사용할 것을 권장합니다.

### Common non-parametrized gates (Clifford)


In [11]:
from qiskit.opflow import H

operator = H ^ H

print(operator)

     ┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     └───┘


하지만 이제부터는 다음과 같이 코드를 작성할 것을 권장합니다.

In [13]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator

qc = QuantumCircuit(2)
qc.h(0)
qc.h(1)

operator = Operator(qc)

print(qc)

     ┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     └───┘


### 1 Qubit Pauli

알고리듬의 검증에 자주, 그리고 요긴하게 사용되는 1큐비트 파울리 행렬을 사용하여 operator를 정의하는 코드는 기존의 다음과 같은 코드를

In [9]:
from qiskit.opflow import I, X, Z, PauliSumOp

operator = 0.39 * (I ^ Z ^ I) + 0.5 * (I ^ X ^ X)

# equivalent to:
operator = PauliSumOp.from_list([("IZI", 0.39), ("IXX", 0.5)])

print(repr(operator))

PauliSumOp(SparsePauliOp(['IZI', 'IXX'],
              coeffs=[0.39+0.j, 0.5 +0.j]), coeff=1.0)


다음과 같은 세가지의 새로운 방식으로 사용할 것을 권장합니다.

In [10]:
from qiskit.quantum_info import SparsePauliOp

operator = SparsePauliOp(["IZI", "IXX"], coeffs = [0.39, 0.5])

# equivalent to:
operator = SparsePauliOp.from_list([("IZI", 0.39), ("IXX", 0.5)])

# equivalent to:
operator = SparsePauliOp.from_sparse_list([("Z", [1], 0.39), ("XX", [0,1], 0.5)], num_qubits = 3)

print(repr(operator))

SparsePauliOp(['IZI', 'IXX'],
              coeffs=[0.39+0.j, 0.5 +0.j])


## Applying Operator to a state

연산자를 양자상태에 적용하는 방식이 이번 업데이트의 영향을 받습니다.

In [17]:
# 이전 방식
from qiskit.opflow import StateFn, X, Y
from qiskit import QuantumCircuit

qc = QuantumCircuit(2)
qc.x(0)
qc.z(1)
op = X ^ Y
state = StateFn(qc)

comp = ~op @ state
eval = comp.eval()

print(state)
print(comp)
print(repr(eval))

CircuitStateFn(
     ┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ Z ├
     └───┘
)
CircuitStateFn(
     ┌───┐┌────────────┐
q_0: ┤ X ├┤0           ├
     ├───┤│  Pauli(XY) │
q_1: ┤ Z ├┤1           ├
     └───┘└────────────┘
)
VectorStateFn(Statevector([ 0.0e+00+0.j,  0.0e+00+0.j, -6.1e-17-1.j,  0.0e+00+0.j],
            dims=(2, 2)), coeff=1.0, is_measurement=False)


C:\Users\8J5337897\AppData\Local\Temp\ipykernel_25244\762569127.py:9: DeprecationWarning: The class ``qiskit.opflow.state_fns.circuit_state_fn.CircuitStateFn`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  state = StateFn(qc)


In [18]:
# 새로운 방식

from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp, Statevector

qc = QuantumCircuit(2)
qc.x(0)
qc.z(1)
op = SparsePauliOp("XY")
state = Statevector(qc)

eval = state.evolve(op)

print(state)
print(eval)

Statevector([0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
            dims=(2, 2))
Statevector([0.+0.j, 0.+0.j, 0.-1.j, 0.+0.j],
            dims=(2, 2))


## Trotter Evolution

In [19]:
# 이전방식 - Opflow

from qiskit.opflow import Trotter, PauliTrotterEvolution, PauliSumOp

hamiltonian = PauliSumOp.from_list([('X', 1), ('Z',1)])
evolution = PauliTrotterEvolution(trotter_mode=Trotter(), reps=2)
evol_result = evolution.convert(hamiltonian.exp_i())
evolved_state = evol_result.to_circuit()

print(evolved_state)

   ┌─────────────────────┐
q: ┤ exp(-it (X + Z))(1) ├
   └─────────────────────┘


C:\Users\8J5337897\AppData\Local\Temp\ipykernel_25244\1305101551.py:6: DeprecationWarning: The class ``qiskit.opflow.evolutions.trotterizations.trotter.Trotter`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  evolution = PauliTrotterEvolution(trotter_mode=Trotter(), reps=2)
C:\Users\8J5337897\AppData\Local\Temp\ipykernel_25244\1305101551.py:6: DeprecationWarning: The class ``qiskit.opflow.evolutions.pauli_trotter_evolution.PauliTrotterEvolution`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  evolution = PauliTrotterEvolution(trotter_mode=Trotter(), reps=2)


In [20]:
# 새로운 방식 - qiskit.synthesis

from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.synthesis import SuzukiTrotter

hamiltonian = SparsePauliOp.from_list([('X', 1), ('Z',1)])
evol_gate = PauliEvolutionGate(hamiltonian, time=1, synthesis=SuzukiTrotter(reps=2))
evolved_state = QuantumCircuit(1)
evolved_state.append(evol_gate, [0])

print(evolved_state)

   ┌─────────────────────┐
q: ┤ exp(-it (X + Z))(1) ├
   └─────────────────────┘


## Evolution with time-dependent Hamiltonian

In [21]:
# 이전방식 - opflow

from qiskit.opflow import Trotter, PauliTrotterEvolution, PauliSumOp
from qiskit.circuit import Parameter

time = Parameter('t')
hamiltonian = PauliSumOp.from_list([('X', 1), ('Y',1)])
evolution = PauliTrotterEvolution(trotter_mode=Trotter(), reps=1)
evol_result = evolution.convert((time * hamiltonian).exp_i())
evolved_state = evol_result.to_circuit()

print(evolved_state)

   ┌─────────────────────────┐
q: ┤ exp(-it (X + Y))(1.0*t) ├
   └─────────────────────────┘


C:\Users\8J5337897\AppData\Local\Temp\ipykernel_25244\2538675533.py:8: DeprecationWarning: The class ``qiskit.opflow.evolutions.trotterizations.trotter.Trotter`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  evolution = PauliTrotterEvolution(trotter_mode=Trotter(), reps=1)
C:\Users\8J5337897\AppData\Local\Temp\ipykernel_25244\2538675533.py:8: DeprecationWarning: The class ``qiskit.opflow.evolutions.pauli_trotter_evolution.PauliTrotterEvolution`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  evolution = PauliTrotterEvolution(trotter_mode=Trotter(), reps=1)


In [22]:
# 새로운 방식 - Qiskit synthesis

from qiskit.quantum_info import SparsePauliOp
from qiskit.synthesis import LieTrotter
from qiskit.circuit.library import PauliEvolutionGate
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter

time = Parameter('t')
hamiltonian = SparsePauliOp.from_list([('X', 1), ('Y',1)])
evol_gate = PauliEvolutionGate(hamiltonian, time=time, synthesis=LieTrotter())
evolved_state = QuantumCircuit(1)
evolved_state.append(evol_gate, [0])

print(evolved_state)

   ┌─────────────────────┐
q: ┤ exp(-it (X + Y))(t) ├
   └─────────────────────┘


이와 관련한 더 많은 변경 사항과 새로운 예제 코드는 [Opflow Migration Guide](https://qiskit.org/documentation/migration_guides/opflow_migration.html)에서 확인하실 수 있습니다.

# Quantum Instance

Quantum Instance는 양자 회로를 백엔드에 실행할때 유용하게 사용되던 기능이었습니다. 새로운 사용법과 관련한 코드는 다음과 같습니다.

## Circuit Sampling

In [23]:
# quantuminstance

from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.utils import QuantumInstance

circuit = QuantumCircuit(2)
circuit.x(0)
circuit.x(1)
circuit.measure_all()

simulator = AerSimulator()
qi = QuantumInstance(backend=simulator, shots=200)
result = qi.execute(circuit).results[0]
data = result.data
counts = data.counts

print("Counts: ", counts)
print("Data: ", data)
print("Result: ", result)

Counts:  {'0x3': 200}
Data:  ExperimentResultData(counts={'0x3': 200})
Result:  ExperimentResult(shots=200, success=True, meas_level=2, data=ExperimentResultData(counts={'0x3': 200}), header=QobjExperimentHeader(creg_sizes=[['meas', 2]], global_phase=0.0, memory_slots=2, metadata=None, n_qubits=2, name='circuit-173', qreg_sizes=[['q', 2]]), status=DONE, seed_simulator=325188135, metadata={'batched_shots_optimization': False, 'method': 'stabilizer', 'active_input_qubits': [0, 1], 'device': 'CPU', 'remapped_qubits': False, 'num_qubits': 2, 'num_clbits': 2, 'sample_measure_time': 0.0002268, 'input_qubit_map': [[0, 0], [1, 1]], 'measure_sampling': True, 'noise': 'ideal', 'parallel_shots': 1, 'parallel_state_update': 16, 'fusion': {'enabled': False}}, time_taken=0.00069)


C:\Users\8J5337897\AppData\Local\Temp\ipykernel_25244\2483793867.py:13: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qi = QuantumInstance(backend=simulator, shots=200)


In [24]:
from qiskit import QuantumCircuit
from qiskit.primitives import Sampler

circuit = QuantumCircuit(2)
circuit.x(0)
circuit.x(1)
circuit.measure_all()

sampler = Sampler()
result = sampler.run(circuit, shots=200).result()
quasi_dists = result.quasi_dists

print("Quasi-dists: ", quasi_dists)
print("Result: ", result)

Quasi-dists:  [{3: 1.0}]
Result:  SamplerResult(quasi_dists=[{3: 1.0}], metadata=[{'shots': 200}])


## Circuit Sampling with Error mitigation

In [27]:
#quantuminstance

from qiskit import QuantumCircuit
from qiskit.utils import QuantumInstance
from qiskit.utils.mitigation import CompleteMeasFitter
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_provider import IBMProvider


circuit = QuantumCircuit(2)
circuit.x(0)
circuit.x(1)
circuit.measure_all()

# Define Quantum Instance with noisy simulator
provider = IBMProvider()
device = provider.get_backend("ibm_nairobi")
noise_model = NoiseModel.from_backend(device)
coupling_map = device.configuration().coupling_map

backend = AerSimulator()

qi = QuantumInstance(
    backend=backend,
    shots=4000,
    measurement_error_mitigation_cls=CompleteMeasFitter,
    cals_matrix_refresh_period=0,
)

result = qi.execute(circuit).results[0].data
print(result)

ExperimentResultData(counts={'11': 4000})


C:\Users\8J5337897\AppData\Local\Temp\ipykernel_25244\535179200.py:23: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qi = QuantumInstance(


In [28]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer.noise import NoiseModel
from qiskit_aer.primitives import Estimator
from qiskit_ibm_provider import IBMProvider

# Define problem
op = SparsePauliOp("XY")
qc = QuantumCircuit(2)
qc.x(0)
qc.x(1)

# Define Aer Estimator with noisy simulator
device = provider.get_backend("ibm_nairobi")
noise_model = NoiseModel.from_backend(device)
coupling_map = device.configuration().coupling_map

# if Noise Model provided, the aer primitives
# perform a "qasm" simulation
estimator = Estimator(
           backend_options={ # method chosen automatically to match options
               "coupling_map": coupling_map,
               "noise_model": noise_model,
           },
           run_options={"seed": 42, "shots": 1024},
          transpile_options={"seed_transpiler": 42},
       )

# Run
expectation_value = estimator.run(qc, op).result().values

print(expectation_value)

[-0.0390625]


더 많은 내용은 [QuantumInstance Migration Guide](https://qiskit.org/documentation/migration_guides/qi_migration.html)를 참고해 주세요.

# 알고리듬 구현 관련

위의 업데이트들은 알고리듬 구현과 관련하여 여러가지 변경사항을 포함합니다. 아래의 예제 코드는 VQE, QAOA 구현에 있어서의 코드 변경사항에 대한 예제입니다.

## VQE

In [29]:
#이전 방식 - quantum instnace

from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import SPSA
from qiskit.circuit.library import TwoLocal
from qiskit.opflow import PauliSumOp
from qiskit.utils import QuantumInstance
from qiskit_aer import AerSimulator

ansatz = TwoLocal(2, 'ry', 'cz')
opt = SPSA(maxiter=50)

# shot-based simulation
backend = AerSimulator()
qi = QuantumInstance(backend=backend, shots=2048, seed_simulator=42)
vqe = VQE(ansatz, optimizer=opt, quantum_instance=qi)

hamiltonian = PauliSumOp.from_list([("XX", 1), ("XY", 1)])
result = vqe.compute_minimum_eigenvalue(hamiltonian)

print(result.eigenvalue)

C:\Users\8J5337897\AppData\Local\Temp\ipykernel_25244\3039800041.py:15: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qi = QuantumInstance(backend=backend, shots=2048, seed_simulator=42)
C:\Users\8J5337897\AppData\Local\Temp\ipykernel_25244\3039800041.py:16: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.vqe.VQE`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.VQE``. See https://qisk.it/algo_migration for a migration guide.
  vqe = VQE(ansatz, optimizer=opt, quantum_instance=qi)


(-1.037109375+0j)


In [31]:
# 새방식 - Estimator Primitive

from qiskit.algorithms.minimum_eigensolvers import VQE  # new import!!!
from qiskit.algorithms.optimizers import SPSA
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator
from qiskit_aer.primitives import Estimator as AerEstimator

ansatz = TwoLocal(2, 'ry', 'cz')
opt = SPSA(maxiter=50)

# shot-based simulation
estimator = Estimator(options={"shots": 2048})
vqe = VQE(estimator, ansatz, opt)

# another option
aer_estimator = AerEstimator(run_options={"shots": 2048, "seed": 42})
vqe = VQE(aer_estimator, ansatz, opt)

hamiltonian = SparsePauliOp.from_list([("XX", 1), ("XY", 1)])
result = vqe.compute_minimum_eigenvalue(hamiltonian)

print(result.eigenvalue)

-0.9697265625


## QAOA

In [32]:
#이전방식 
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA
from qiskit.opflow import PauliSumOp
from qiskit.utils import QuantumInstance
from qiskit_aer import AerSimulator

# exact statevector simulation
backend = AerSimulator()
qi = QuantumInstance(backend=backend, shots=None,
        seed_simulator = 42, seed_transpiler = 42,
        backend_options={"method": "statevector"})

optimizer = COBYLA()
qaoa = QAOA(optimizer=optimizer, reps=2, quantum_instance=qi)

# diagonal operator
qubit_op = PauliSumOp.from_list([("ZIII", 1),("IZII", 1), ("IIIZ", 1), ("IIZI", 1)])
result = qaoa.compute_minimum_eigenvalue(qubit_op)

print(result.eigenvalue.real)

C:\Users\8J5337897\AppData\Local\Temp\ipykernel_25244\3924945156.py:10: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qi = QuantumInstance(backend=backend, shots=None,
C:\Users\8J5337897\AppData\Local\Temp\ipykernel_25244\3924945156.py:15: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.qaoa.QAOA`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.QAOA``. See https://qisk.it/algo_migration for a migration guide.
  qaoa = QAOA(optimizer=optimizer, reps=2, quantum_instance=qi)


-3.99609375


In [33]:
#새로운방식 - Sampler Primitive

from qiskit.algorithms.minimum_eigensolvers import QAOA
from qiskit.algorithms.optimizers import COBYLA
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Sampler
from qiskit_aer.primitives import Sampler as AerSampler

# exact statevector simulation
sampler = Sampler()

# another option
sampler = AerSampler(backend_options={"method": "statevector"},
                     run_options={"shots": None, "seed": 42})

optimizer = COBYLA()
qaoa = QAOA(sampler, optimizer, reps=2)

# diagonal operator
qubit_op = SparsePauliOp.from_list([("ZIII", 1),("IZII", 1), ("IIIZ", 1), ("IIZI", 1)])
result = qaoa.compute_minimum_eigenvalue(qubit_op)

print(result.eigenvalue)

-3.9999999560279926


Qiskit Global 커뮤니티는 2주에 한번씩, 개발자 밋업을 개최하고 있습니다.

- Repo: https://github.com/Qiskit/feedback/wiki/Qiskit-DemoDays
- Tutorial Notebook: https://github.com/Qiskit/feedback/tree/main/demo-day-notebooks

감사합니다!